**Week:** 11

**Assignment from Group:** Uptight Wealth

**Link:** https://github.com/Micniks/Python-Week11-Group-3-Assignments

# The Python Cult

## Assignment 1: Make a Cultist Model
1. import the data from the dataset, and store it in a pandas variable ***cult***
2. Divide the data into two datasets, ***cultists*** and ***roles***. Make the X dataset for the stats(*features*) of the cultists, and the Y dataset for the position(*targets*) for the dataset.
3. Refactor the dataset ***cultists***, by removing the irrelevent features from in the dataset, so it only contains charactistic statistics
```python
['stealth', 'Influence', 'Endurance', 'Lore', 'Economic', 'Strength', 'Insanity']
```
4. Refactor the dataset ***roles*** so each value become machine friendly numbers, instead of strings.
5. Use LinearRegression to make a model object.
6. Use train_test_split from model_selection, to make training and testing dataset from ***cultists*** and ***roles***
   1. Try adjusting the test_size variable later, to see what gives the better scores.
7. Feed the model with the training datasets, and score the model on the training dataset (*model.score*)
8. Score the model on the testing datasets, and compare to the testing scores.

In [31]:
import pandas as pd
import sklearn.linear_model
from sklearn.model_selection import train_test_split

#1.1
data = pd.read_csv('https://raw.githubusercontent.com/Micniks/Python-Week11-Group-3-Assignments/main/cultists.csv')
#display(data)

#1.2, 1.3
xs = data[['Stealth', 'Influence', 'Endurance', 'Lore', 'Economic', 'Strength', 'Insanity']]
ys = data[['Position']]

#display(xs)
#display(ys)

label_mapping = {
    'Priest':0, 
    'Enforcer':1, 
    'Assassin':2, 
    'Recruiter':3,
    'Accountant':4,
    'Advisor':5
}

ys['Position'] = ys['Position'].map(label_mapping)
#display(ys)

print(xs.shape)
print(ys.shape)

#1.4
model = sklearn.linear_model.LinearRegression()

#1.5
x_train,x_test,y_train,y_test = train_test_split(xs,ys,test_size=0.50)

model.fit(x_train, y_train)

print('Training result score:', model.score(x_train,y_train))
print('Testing result score:', model.score(x_test,y_test))

(6666, 7)
(6666, 1)
Training result score: 0.5045329220826917
Testing result score: 0.4944116486454459


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


## Assignment 2: Classification of Cultists
*Since the cultist data doesn't seem to have any linear progression, another model might give better result in sorting members...*

1. Use DecisionTreeClassifier to make a new model object
2. Use train_test_split from model_selection, to make training and testing dataset from ***cultists*** and ***roles***
   1. Try adjusting the test_size variable later, to see what gives the better scores.
3. Feed the model with the training datasets, and score the model on the training dataset (*accuracy _score from sklearn.metrics*)
4. Score the model on the testing datasets, and compare to the testing scores.
5. Compare the scores from using Classification and using Regression models.

*We suspect the score difference is the result of the datasets structure not progressing in a linear fashion*

In [64]:
from sklearn.tree import DecisionTreeClassifier

#2.1
#model = DecisionTreeClassifier(max_leaf_nodes=500, random_state=0)
model = DecisionTreeClassifier()

#2.2 Irrelevent
x_train,x_test,y_train,y_test = train_test_split(xs,ys,test_size=0.3)

#2.3
model.fit(x_train, y_train)
print('Train Score:', model.score(x_train, y_train))

#2.4
print('Testing Score',model.score(x_test, y_test))

#2.5
#As the data does not have a linear structur, a model will be more accute while using Classification over Regression

Train Score: 1.0
Testing Score 0.7195


## Assignment 3: Find Clusters of Cultists
1. Use the orignal dataset again, and remove the charactistic statistics used earlier, as well as the name and position for each member.
2. Change the Living_Area feature into multiple numeric features, using One Hot Encoding
3. Remove all rows with missing data for any features
4. Make a model/analyzer from sklearn.cluster, with the appropriate bandwidth for the data.
5. Use the analyzer to process the data, grouping people into clusters.
6. Make a dataset from the cluster-array, showing the avarage statistic values for features in each cluster
7. Add a count feature to each cluster, showing how many members are in each cluster
8. Look at the final data, and answer the following questions:
   1. How many clusters are there
   2. Which cluster has the highest amount of recruits
   3. What seems to be the defining feature(s) for each cluster

In [94]:
from sklearn.cluster import estimate_bandwidth
from sklearn.cluster import MeanShift
import numpy as np
import pandas as pd

max_bandwidth = 300
min_cluster_size = 10

#3.1
c_data = data.drop(['Cultist', 'Stealth', 'Influence', 'Endurance', 'Lore', 'Economic', 'Strength', 'Insanity', 'Position'], axis=1)
#display(c_data)

#3.3 REVERSE WITH 3.2
c_data = c_data.dropna()
#display(c_data)

#3.2 REVERSE WITH 3.3
c_data = pd.get_dummies(c_data,columns=['Type_of_living_area'])
#display(c_data)

#3.4
bandwidth_size = round(estimate_bandwidth(c_data),0)
print('Estimated needed Bandwidth size:', bandwidth_size)
if(bandwidth_size > max_bandwidth):
    print('Estimated Bandwidth too large, swiching to max bandwidth of:', max_bandwidth)
    bandwidth_size = max_bandwidth
analyzer = MeanShift(bandwidth=bandwidth_size) 
analyzer.fit(c_data)

labels = analyzer.labels_
print('Amount of Clusters:',len(np.unique(labels)))

c_data['cluster_group'] = np.nan
for i in range(len(c_data)):
    c_data.iloc[i,c_data.columns.get_loc('cluster_group')] = labels[i]
    
#Grouping cultists by Cluster
c_cluster_data = c_data.groupby(['cluster_group']).mean()

#Count of cultists in each cluster
c_cluster_data['Counts'] = pd.Series(c_data.groupby(['cluster_group']).size())

c_cluster_result_data = c_cluster_data[(c_cluster_data['Counts'] >= min_cluster_size)]
c_cluster_discard_data = c_cluster_data[(c_cluster_data['Counts'] < min_cluster_size)]

print('Minimum Cluster sizes for Results:', min_cluster_size)
print('Amount of Clusters with less then minimum size:', len(c_cluster_discard_data))

print('\nResulting Cluster Amount:', len(c_cluster_result_data))

display(c_cluster_result_data)

Estimated needed Bandwidth size: 1900.0
Estimated Bandwidth too large, swiching to max bandwidth of: 300
Amount of Clusters: 35
Minimum Cluster sizes for Results: 10
Amount of Clusters with less then minimum size: 9

Resulting Cluster Amount: 26


,Age,Membership_in_years,Contribution_in_dollars,Members_recruited,Type_of_living_area_City,Type_of_living_area_Province,Type_of_living_area_Rural,Counts
cluster_group,,,,,,,,
0.0,37.927329,7.931056,409.625466,7.703727,0.609317,0.244099,0.146584,1610
1.0,43.368941,15.510772,1291.470377,16.771993,0.589767,0.249551,0.160682,1114
2.0,46.542805,18.577413,2047.735883,18.267760,0.612022,0.220401,0.167577,549
3.0,48.342975,20.814050,2614.900826,18.394628,0.547521,0.268595,0.183884,484
4.0,50.532688,23.348668,3194.929782,25.847458,0.578692,0.234867,0.186441,413
5.0,49.794872,24.166667,3763.641026,27.557692,0.618590,0.250000,0.131410,312
6.0,52.622150,25.807818,4305.517915,28.465798,0.589577,0.250814,0.159609,307
7.0,51.856522,27.295652,4835.486957,18.191304,0.617391,0.239130,0.143478,230
8.0,53.349673,29.398693,5446.395425,31.323529,0.588235,0.238562,0.173203,306
